In [1]:
import sympy as sp

from symlib.utils import pull_sums_out_front, sum_kronecker_contract, wilds, wild_subs
from symlib.random import ExpVal, GaussianIndexedBase, wick_contraction
from symlib.MLP import ZLayer

In [2]:
# Indices
i1, i2, i3, i4 = sp.symbols('i1 i2 i3 i4', integer=True)
alpha1, alpha2, alpha3, alpha4 = sp.symbols('alpha1, alpha2, alpha3, alpha4', integer=True)

# Input
x = sp.IndexedBase('x')
n_0 = sp.symbols('n_0', integer=True)

# First layer param
b = GaussianIndexedBase('b^(1)')
W = GaussianIndexedBase('W^(1)')
C_b, C_w = sp.symbols('C_b^(1) C_W^(1)', positive=True)

## Two point correlator

In [3]:
z1 = ZLayer(W, b, x, n_0)

z1[i1, alpha1] * z1[i2, alpha2]

(b^(1)[i1] + Sum(W^(1)[i1, k_1]*x[k_1, alpha1], (k_1, 1, n_0)))*(b^(1)[i2] + Sum(W^(1)[i2, k_2]*x[k_2, alpha2], (k_2, 1, n_0)))

In [4]:
zz: sp.Basic = sp.expand(z1[i1, alpha1] * z1[i2, alpha2])
zz = pull_sums_out_front(zz)
zz

b^(1)[i1]*b^(1)[i2] + Sum(W^(1)[i1, k_3]*b^(1)[i2]*x[k_3, alpha1], (k_3, 1, n_0)) + Sum(W^(1)[i2, k_4]*b^(1)[i1]*x[k_4, alpha2], (k_4, 1, n_0)) + Sum(W^(1)[i1, k_3]*W^(1)[i2, k_4]*x[k_3, alpha1]*x[k_4, alpha2], (k_3, 1, n_0), (k_4, 1, n_0))

In [5]:
Ezz: sp.Basic = ExpVal(zz)
Ezz

𝔼[b^(1)[i1]*b^(1)[i2]] + Sum(𝔼[W^(1)[i1, k_3]*b^(1)[i2]]*x[k_3, alpha1], (k_3, 1, n_0)) + Sum(𝔼[W^(1)[i2, k_4]*b^(1)[i1]]*x[k_4, alpha2], (k_4, 1, n_0)) + Sum(𝔼[W^(1)[i1, k_3]*W^(1)[i2, k_4]]*x[k_3, alpha1]*x[k_4, alpha2], (k_3, 1, n_0), (k_4, 1, n_0))

In [6]:
A, B, C, D = wilds('A, B, C, D')

gaussian_init_rules = {
    ExpVal(W[A, B] * b[C]): 0,
    ExpVal(b[A] * b[B]): C_b * sp.KroneckerDelta(A, B),
    ExpVal(W[A, B] * W[C, D]): C_w * sp.KroneckerDelta(A, C) * sp.KroneckerDelta(B, D)/n_0
}

Ezz = wild_subs(Ezz, gaussian_init_rules)
Ezz

C_b^(1)*KroneckerDelta(i1, i2) + Sum(0, (k_3, 1, n_0)) + Sum(0, (k_4, 1, n_0)) + Sum(C_W^(1)*KroneckerDelta(i1, i2)*KroneckerDelta(k_3, k_4)*x[k_3, alpha1]*x[k_4, alpha2]/n_0, (k_3, 1, n_0), (k_4, 1, n_0))

In [7]:
Ezz = sum_kronecker_contract(Ezz).doit()
Ezz

C_b^(1)*KroneckerDelta(i1, i2) + Sum(C_W^(1)*KroneckerDelta(i1, i2)*x[k_3, alpha1]*x[k_3, alpha2]/n_0, (k_3, 1, n_0))

## First-layer metric

In [8]:
G_a1a2 = sp.simplify(Ezz)/sp.KroneckerDelta(i1,i2)
G_a1a2

C_W^(1)*Sum(x[k_3, alpha1]*x[k_3, alpha2], (k_3, 1, n_0))/n_0 + C_b^(1)

In [9]:
z1.gaussian_init(C_w, C_b)
G1 = z1.layer_metric
G1[alpha1, alpha2]

C_W^(1)*Sum(x[k_5, alpha1]*x[k_5, alpha2], (k_5, 1, n_0))/n_0 + C_b^(1)

# Four point correlator

In [10]:
z = GaussianIndexedBase('z^(1)')
G = GaussianIndexedBase('G^(1)')
zzzz = z[i1, alpha1] * z[i2, alpha2] * z[i3, alpha3] * z[i4, alpha4]
zzzz

z^(1)[i1, alpha1]*z^(1)[i2, alpha2]*z^(1)[i3, alpha3]*z^(1)[i4, alpha4]

In [11]:
Ezzzz: sp.Basic = ExpVal(pull_sums_out_front(sp.expand(zzzz)))
Ezzzz

𝔼[z^(1)[i1, alpha1]*z^(1)[i2, alpha2]*z^(1)[i3, alpha3]*z^(1)[i4, alpha4]]

In [12]:
Ezzzz = wick_contraction(Ezzzz)
Ezzzz

𝔼[z^(1)[i1, alpha1]*z^(1)[i2, alpha2]]*𝔼[z^(1)[i3, alpha3]*z^(1)[i4, alpha4]] + 𝔼[z^(1)[i1, alpha1]*z^(1)[i3, alpha3]]*𝔼[z^(1)[i2, alpha2]*z^(1)[i4, alpha4]] + 𝔼[z^(1)[i1, alpha1]*z^(1)[i4, alpha4]]*𝔼[z^(1)[i2, alpha2]*z^(1)[i3, alpha3]]

In [13]:
Ezzzz.replace(ExpVal(z[A, B] * z[C, D]), sp.KroneckerDelta(A, C) * G[B, D])

KroneckerDelta(i1, i2)*KroneckerDelta(i3, i4)*G^(1)[alpha1, alpha2]*G^(1)[alpha3, alpha4] + KroneckerDelta(i1, i3)*KroneckerDelta(i2, i4)*G^(1)[alpha1, alpha3]*G^(1)[alpha2, alpha4] + KroneckerDelta(i1, i4)*KroneckerDelta(i2, i3)*G^(1)[alpha1, alpha4]*G^(1)[alpha2, alpha3]